In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import *


import tensorflow as tf
import keras.backend as K
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras import datasets
from keras.wrappers.scikit_learn import KerasClassifier


import matplotlib.pyplot as plt
import time

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# LEITURA DA BASE

In [2]:
df = pd.read_csv("Tweets.csv", usecols = ["text", "airline_sentiment"]) #usando apenas as colunas necessárias para o problema
#Renomeando a coluna da classe
df.rename(columns = {"airline_sentiment" : "category"}, inplace = True)

_____

# ANÁLISE EXPLORATÓRIA

* 1 - Verifica tamanho da base
* 2 - Verifica quantidade de linhas duplicadas
* 3 - Verifica quantidade de dados nulos
* 4 - Verifica quantidade de observações para cada categoria

In [3]:
# 1 - Tamanho da base
print("Tamanho da base original (quantidade de tweets / reviews:", df.shape[0])

Tamanho da base original (quantidade de tweets / reviews: 14640


In [4]:
# 2 - Verifica quantidade de linhas duplicadas
df.duplicated().sum()

188

In [5]:
## Removendo linhas duplicadas

duplicated_index = list(df[df.duplicated()].index)
df.drop(duplicated_index, axis = 0, inplace = True)
print("Tamanho da base sem linhas duplicadas:", df.shape[0])

Tamanho da base sem linhas duplicadas: 14452


In [6]:
# 3 - Verifica quantidade de dados nulos
df.isnull().sum()

category    0
text        0
dtype: int64

In [7]:
# 4 - Verifica quantidade de observações para cada categoria

df.category.value_counts()

negative    9087
neutral     3067
positive    2298
Name: category, dtype: int64

____

# PRÉ-PROCESSAMENTO DOS TEXTOS

In [8]:
def tweet_to_words(tweet):
    ''' Converte o texto em uma sequência (lista) de palavras'''

    # converte as letras das palavras para minúsculo
    text = tweet.lower()

    # remove tickers do mercado de ações como $GE
    text = re.sub(r'\$\w*', '', text)
    # o texto de retuíte de estilo antigo "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks
    text = re.sub(r'https?://\S+', '', text)

    # remove o sinal de hash # da palavra e a marcação (@...)
    #text = re.sub(r'#', '', tweet)
    text = re.sub(r'@(\w+)', '', text)

    # remove caracteres especiais
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # converte o texto em lista de palavras onde cada palavra é chamada de token
    words = text.split()
    # remove stopwords (palavras que costumama aparecer com frequencia e não são relevantes no texto)
    stopwords_list = stopwords.words("english")
    #Lista de stopwords que serão mantidas e que podem ser relevantes para o problema
    keep_stopwords = ['against', 'below', 'up', 'down', 'in','out','off','over' ,'under' ,'all' ,'any' ,'few' ,'more' ,'most' ,'such' ,'no' ,'not' ,'only' ,'very',
                  'again','both','nor','very','too', "don't", "didn't", "couldn't", "doesn't", "isn't", "won't", "wouldn't", "wasn't", "weren't", "aren't"]
    #Removendo essas palavras da lista original de stopwords
    stopwords_list_final = [x for x in stopwords_list if x not in keep_stopwords]

    words = [w for w in words if w not in stopwords_list_final]

    # aplica stemming (converte as palavras para seu radical para diminuir o vocabulário)
    #Ex: banda, bandas --> band

    words = [PorterStemmer().stem(w) for w in words]

    # returna a lista de palavras tokenizadas
    return words



# APLICANDO O PRÉ-PROCESSAMENTO EM CADA COLUNA DO DATAFRAME

preprocessed_texts = list(map(tweet_to_words, df['text']))

print("Texto original")
print(df.iloc[250].text)
print("\n")
print("Texto tratado e tokenizado")
print(preprocessed_texts[250])

Texto original
@VirginAmerica current bug on website shows ‘select departure city’ when selecting destination city http://t.co/SLLYIBE2vQ


Texto tratado e tokenizado
['current', 'bug', 'websit', 'show', 'select', 'departur', 'citi', 'select', 'destin', 'citi']


In [10]:
# # Conversão dos tokens de palavras para tokens/vetores com valores inteiros

max_words = 5000 #número máximo de palavras que serão mantidas com base em sua frequência
max_len = 50 #tamanho máximo do vetor de inteiros para cada texto

def tokenize_pad_sequences(text):

    # Tokenização do texto em palavras individuais
    tokenizer = Tokenizer(num_words = max_words, lower=True, split=' ') #num_words indica a qntde de palaras que serão mantidas
    tokenizer.fit_on_texts(text) #treina o tokenizador e cria o vocabulário interno (com max_words palavras)

    # Converte o texto em uma sequência de valores inteiros
    X = tokenizer.texts_to_sequences(text)
    # Cada palavra do texto é substituída por um número inteiro correspondente ao seu índice no vocabulário
    #construído pelo tokenizer.

    # O Padding faz com que todos os vetores de inteiros de cada palavra tenham a mesma dimensão
    X = pad_sequences(X, padding='post', maxlen = max_len) #preenche os vetores para que todos eles tenham o mesmo tamanho


    return X, tokenizer

print('Antes da tokenização e padding \n', preprocessed_texts[250])
X, tokenizer = tokenize_pad_sequences(preprocessed_texts)
print("\n")
print('Depois da tokenização e padding \n', X[250])

Antes da tokenização e padding 
 ['current', 'bug', 'websit', 'show', 'select', 'departur', 'citi', 'select', 'destin', 'citi']


Depois da tokenização e padding 
 [ 411 1681  178  190  629  267  452  629  394  452    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


_____

# MODELAGEM

## SEPARAÇÃO DOS DADOS DE TREINO, VALIDAÇÃO E TESTE

In [11]:
#Convertendo a categoria em variáveis dummy (one-hot encoding)
y = pd.get_dummies(df['category'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print('Conjunto de treino ->', X_train.shape, y_train.shape)
print('Conjunto de validação ->', X_val.shape, y_val.shape)
print('Conjunto de teste ->', X_test.shape, y_test.shape)

Conjunto de treino -> (9248, 50) (9248, 3)
Conjunto de validação -> (2313, 50) (2313, 3)
Conjunto de teste -> (2891, 50) (2891, 3)


## LSTM Bidirecional

In [13]:
# Hiperparâmetros da LSTM
vocab_size = 5000
embedding_size = 50
learning_rate = 0.1
momentum = 0.8

# Otimizador
sgd = SGD(lr=learning_rate, momentum=momentum)

# Função para criar o modelo
def create_model(dropout=0.4):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=max_len))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dropout(dropout))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model


# Definir os hiperparâmetros a serem ajustados
param_grid = {
    'epochs': [40,50],
    'dropout': [0.4,0.6]
}

# Criar o objeto KerasClassifier
keras_model = KerasClassifier(build_fn=create_model, verbose=1)

# Criar o objeto GridSearchCV
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3, scoring='accuracy')

start = time.time()

# Realizar a busca em grid
grid_result = grid.fit(X_train, y_train)

# Obter o melhor modelo e os melhores hiperparâmetros encontrados
best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

# Treinar o melhor modelo com os melhores hiperparâmetros
best_model.fit(X_train, y_train)

# # Fazer previsões nos dados de teste
# y_pred = best_model.predict(X_test)

# # Calcular a acurácia do modelo
# accuracy = accuracy_score(y_test, y_pred)

end = time.time()

# Imprimir os resultados
print("Melhor combinação de hiperparâmetros:", best_params)
#print("Acurácia dos dados de teste:", accuracy)
print('Tempo de processamento:', end - start, 'segundos')

Epoch 1/40
193/193 [==============================] - 12s 31ms/step - loss: 0.8851 - accuracy: 0.6326
Epoch 2/40
193/193 [==============================] - 7s 38ms/step - loss: 0.7340 - accuracy: 0.6811
Epoch 3/40
193/193 [==============================] - 6s 33ms/step - loss: 0.6211 - accuracy: 0.7470
Epoch 4/40
193/193 [==============================] - 7s 36ms/step - loss: 0.5545 - accuracy: 0.7822
Epoch 5/40
193/193 [==============================] - 6s 31ms/step - loss: 0.5080 - accuracy: 0.8026
Epoch 6/40
193/193 [==============================] - 7s 35ms/step - loss: 0.4576 - accuracy: 0.8235
Epoch 7/40
193/193 [==============================] - 6s 30ms/step - loss: 0.3965 - accuracy: 0.8526
Epoch 8/40
193/193 [==============================] - 7s 37ms/step - loss: 0.3596 - accuracy: 0.8641
Epoch 9/40
193/193 [==============================] - 6s 31ms/step - loss: 0.3153 - accuracy: 0.8843
Epoch 10/40
193/193 [==============================] - 7s 37ms/step - loss: 0.2849 - accur

### AVALIAÇÃO DO MODELO LSTM

In [14]:
# Evaluate model on the test set
#loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
# Evaluate model on the test set
_, accuracy = best_model.model.evaluate(X_test, y_test, verbose=0)

print('')
print('Acurácia de teste : {:.4f}'.format(accuracy))


Acurácia de teste : 0.7572


## ÁRVORE DE DECISÃO

In [15]:
# Função que roda o modelo e retorna o melhor resultado, melhor combinação de hiperparâmetros, tempo de execução do modelo e do grid search

def results(model, params, X, y):
    """Retorna: melhores parametros, maior valor de acurácia de teste, tempo de treinamento do modelo,
    tempo de execução do GridSearchCV"""

    start = time.time()

    grid_search = GridSearchCV(model, params, cv=5, scoring="accuracy", return_train_score=True)

    best_model = grid_search.fit(X, y)

    end = time.time()

    # Obter os resultados da validação cruzada
    cv_results = grid_search.cv_results_

    # Obter o número de modelos testados
    num_models_executed = len(cv_results['mean_test_score'])

    process_time = end - start

    return best_model, grid_search.best_params_, round(grid_search.best_score_, 3), num_models_executed, round(process_time, 3)

In [16]:
dt_params = {
    'max_depth': [3, 5, 7, 10, 15],
    'min_samples_split': [1, 2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'class_weight': [
        {'negative': 5, 'positive': 10, 'neutral': 7},
        {'negative': 6, 'positive': 10, 'neutral': 8},
        {'negative': 7, 'positive': 10, 'neutral': 8},
        None
    ]
}

dt = DecisionTreeClassifier(random_state=4)

best_model, best_param, best_score, num_models_executed, grid_time = results(dt, dt_params, X_train, y_train)

y_pred_dt = best_model.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)

print("Melhor combinação de hiperparâmetros:", best_param)
print("Maior valor de acurácia para dados de treino:", best_score)
print("Número de modelos executados:", num_models_executed)
print("Acurácia dos dados de teste:", round(accuracy_dt,3))
print("Tempo de execução do grid search com validação cruzada:", grid_time, "segundos")


Melhor combinação de hiperparâmetros: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Maior valor de acurácia para dados de treino: 0.586
Número de modelos executados: 500
Acurácia dos dados de teste: 0.576
Tempo de execução do grid search com validação cruzada: 62.926 segundos


## NAIVE BAYES MULTINOMIAL

In [17]:
# Reverter a codificação one-hot para a representação original
y_train['original'] = y.idxmax(axis=1)
y_test['original'] = y.idxmax(axis=1)

In [18]:
nb_params = {'alpha' : [0.1,0.5, 1, 1.5], 'class_prior' : [None, [0.3,1,0.7], [0.5,1.2,0.8], [0.6,1.2,0.7]]}

nb = MultinomialNB()

best_model, best_param, best_score, num_models_executed, grid_time = results(nb, nb_params, X_train,y_train['original'])

y_pred_nb = best_model.predict(X_test)

accuracy_nb = accuracy_score(y_test['original'], y_pred_nb)

print("Melhor combinação de hiperparâmetros:", best_param)
print("Maior valor de acurácia para dados de teste:", best_score)
print("Número de modelos executados:", num_models_executed)
print("Acurácia dos dados de teste:", round(accuracy_nb,3))
print("Tempo de execuçào do grid search com validação cruzada", grid_time, "segundos")

Melhor combinação de hiperparâmetros: {'alpha': 0.1, 'class_prior': [0.5, 1.2, 0.8]}
Maior valor de acurácia para dados de teste: 0.429
Número de modelos executados: 16
Acurácia dos dados de teste: 0.443
Tempo de execuçào do grid search com validação cruzada 4.646 segundos


____

# MELHORIAS

## ÁRVORE DE DECISÃO

In [19]:
#Usando como y o valor da coluna category do dataframe original

#USANDO OS MESMOS HIPERPARAMETROS

best_model, best_param, best_score, num_models_executed, grid_time = results(dt, dt_params, X_train, y_train['original'])

y_pred_dt = best_model.predict(X_test)

accuracy_dt = accuracy_score(y_test['original'], y_pred_dt)

print("Melhor combinação de hiperparâmetros:", best_param)
print("Maior valor de acurácia para dados de treino:", best_score)
print("Número de modelos executados:", num_models_executed)
print("Acurácia dos dados de teste:", round(accuracy_dt,3))
print("Tempo de execução do grid search com validação cruzada:", grid_time, "segundos")

Melhor combinação de hiperparâmetros: {'class_weight': None, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Maior valor de acurácia para dados de treino: 0.667
Número de modelos executados: 500
Acurácia dos dados de teste: 0.671
Tempo de execução do grid search com validação cruzada: 173.333 segundos


## NAIVE BAYES MULTINOMIAL

In [20]:
# X_train antes

len(X_train[0])

50

In [21]:
# Conversão do texto em sequências numéricas
sequences = tokenizer.texts_to_sequences(df['text'])

# Conversão das sequências em matrizes de frequência de atributos
X_nb = tokenizer.sequences_to_matrix(sequences, mode='count')

In [22]:
len(X_nb[0])

5000

In [23]:
# Nova separação de dados
X_train, X_test, y_train, y_test = train_test_split(X_nb, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
print('Conjunto de treino ->', X_train.shape, y_train.shape)
print('Conjunto de validação ->', X_val.shape, y_val.shape)
print('Conjunto de teste ->', X_test.shape, y_test.shape)

Conjunto de treino -> (8670, 5000) (8670, 3)
Conjunto de validação -> (2891, 5000) (2891, 3)
Conjunto de teste -> (2891, 5000) (2891, 3)


In [24]:
# Reverter a codificação one-hot para a representação original
y_train['original'] = y.idxmax(axis=1)
y_test['original'] = y.idxmax(axis=1)

In [25]:
#Executando novamente o Naive Bayes Multinomial

nb_params = {'alpha' : [0.1,0.5, 1, 1.5], 'class_prior' : [None, [0.3,1,0.7], [0.5,1.2,0.8], [0.6,1.2,0.7]]}

nb = MultinomialNB()

best_model, best_param, best_score, num_models_executed, grid_time = results(nb, nb_params, X_train,y_train['original'])

y_pred_nb = best_model.predict(X_test)

accuracy_nb = accuracy_score(y_test['original'], y_pred_nb)

print("Melhor combinação de hiperparâmetros:", best_param)
print("Maior valor de acurácia para dados de teste:", best_score)
print("Número de modelos executados:", num_models_executed)
print("Acurácia dos dados de teste:", round(accuracy_nb,3))
print("Tempo de execuçào do grid search com validação cruzada", grid_time, "segundos")

Melhor combinação de hiperparâmetros: {'alpha': 0.5, 'class_prior': None}
Maior valor de acurácia para dados de teste: 0.727
Número de modelos executados: 16
Acurácia dos dados de teste: 0.731
Tempo de execuçào do grid search com validação cruzada 35.024 segundos
